# 📋⚙️ Automatic Evaluation for LLMs benchmarking in Q/A task
This notebook contains a suite of automatic evaluation metrics used to assess the quality of LLM-generated answers in Arabic across 120 Q/A pairs. The goal is to complement human evaluation with objective, reproducible measurements across several linguistic and semantic dimensions. The following metrics are applied:

1. **Semantic Textual Similarity (STS)**
Measures the overall meaning similarity between two texts using cosine similarity between sentence embeddings (via SentenceTransformer). Useful for capturing high-level semantic equivalence, even when surface wording differs.

2. **BERTScore (Precision / Recall / F1)**
Evaluates semantic similarity between generated answers and gold references using contextual embeddings. Based on token-level comparison using a pre-trained Arabic-compatible model (e.g., xlm-roberta-large). F1 score is typically used as the main indicator, reflecting a balance of precision and recall.

## 📌 Examples on the evaluation

In [1]:
from bert_score import score
from sentence_transformers import SentenceTransformer, util

# Example input
candidate = "الهرم الأكبر في الجيزة هو أشهر معلم ثقافي في مصر."
reference = "أشهر معلم ثقافي في مصر هو الهرم الأكبر الموجود في الجيزة."

# ----- BERTScore -----
P, R, F1 = score([candidate], [reference], lang="ar", model_type="xlm-roberta-large")
print("🔷 BERTScore:")
print(f"Precision: {P[0].item():.4f}")
print(f"Recall:    {R[0].item():.4f}")
print(f"F1:        {F1[0].item():.4f}")

# ----- STS (Cosine Similarity) -----
sts_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

emb_candidate = sts_model.encode(candidate, convert_to_tensor=True)
emb_reference = sts_model.encode(reference, convert_to_tensor=True)

cos_sim = util.cos_sim(emb_candidate, emb_reference).item()
print("\n🔶 STS Cosine Similarity:")
print(f"Similarity Score: {cos_sim:.4f}")


🔷 BERTScore:
Precision: 0.9664
Recall:    0.9684
F1:        0.9674

🔶 STS Cosine Similarity:
Similarity Score: 0.9848


In [2]:
from sentence_transformers import SentenceTransformer, util

# Load a multilingual SBERT model that supports Arabic
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Example Arabic Q/A pairs
gold_answers = [
    "السيارة وسيلة شائعة للنقل",
    "القطار يستخدم بشكل كبير في التنقل بين المدن"
]

predicted_answers = [
    "وسيلة النقل الأكثر استخدامًا هي السيارة",
    "القطار من أشهر وسائل النقل بين المدن"
]

# Compute cosine similarity scores for each pair
for gold, pred in zip(gold_answers, predicted_answers):
    emb_gold = model.encode(gold, convert_to_tensor=True)
    emb_pred = model.encode(pred, convert_to_tensor=True)

    similarity = util.cos_sim(emb_gold, emb_pred).item()
    print(f"Gold: {gold}")
    print(f"Pred: {pred}")
    print(f"Semantic similarity score: {similarity:.4f}\n")

Gold: السيارة وسيلة شائعة للنقل
Pred: وسيلة النقل الأكثر استخدامًا هي السيارة
Semantic similarity score: 0.9350

Gold: القطار يستخدم بشكل كبير في التنقل بين المدن
Pred: القطار من أشهر وسائل النقل بين المدن
Semantic similarity score: 0.9106



# 📊 Excel functions

In [3]:
from openpyxl import load_workbook

def extract_questions_and_answers(file_path, question_col=3, answer_col=5, start_row=3):
    wb = load_workbook(file_path)
    ws = wb.active

    questions = []
    gold_answers = []

    row = start_row  # Start for questions
    while True:
        q_cell = ws.cell(row=row, column=question_col)
        a_cell = ws.cell(row=row, column=answer_col)

        # Stop when we reach an empty question cell
        if not q_cell.value:
            break

        questions.append(q_cell.value)
        gold_answers.append(a_cell.value)
        row += 1

    return questions, gold_answers

def write_answers(file_path, answers, output_col=6, start_row=3):
    wb = load_workbook(file_path)
    ws = wb.active

    for i, answer in enumerate(answers):
        row = start_row + i
        ws.cell(row=row, column=output_col, value=answer)
    wb.save(file_path)

# 🧮 Functions to calculate BERTScore and STS

In [4]:
from bert_score import score

def get_bert_score(candidates, references, lang="ar", model_type="xlm-roberta-large"):
    """
    Calculate BERTScore (Precision, Recall, F1) for a list of candidate and reference sentences.
    
    :param candidates: List of candidate sentences (model outputs).
    :param references: List of reference sentences (gold answers).
    :param lang: Language code (default is Arabic).
    :param model_type: Model type for BERTScore.
    :return: Tuple of (list of precision, list of recall, list of F1 scores).
    """
    P, R, F1 = score(candidates, references, lang=lang, model_type=model_type, verbose=True)
    
    for cand, ref, f1 in zip(candidates, references, F1):
        print(f"Candidate: {cand}")
        print(f"Reference: {ref}")
        print("BERTScore F1:", f1.item())
        print("———")

    return P, R, F1


In [5]:
from sentence_transformers import SentenceTransformer, util

def get_STS_score(candidates, references, model):
    """
    Calculate STS score for the given candidates and references.
    :param candidates: List of candidate sentences (model outputs).
    :param references: List of reference sentences (gold answers).
    :param model: SentenceTransformer model to use for STS calculation.
    :return: STS scores.
    """
    scores = []
    for candidate, reference in zip(candidates, references):
        if not candidate or not reference:
            scores.append(0.0)
            continue
        emb_candidate = model.encode(candidate, convert_to_tensor=True)
        emb_reference = model.encode(reference, convert_to_tensor=True)
        score = util.cos_sim(emb_reference, emb_candidate).item()
        scores.append(score)
        print(f"Candidate: {candidate}")
        print(f"Reference: {reference}")
        print(f"STS score: {score:.4f}\n")
    return scores

# 🖩 Calculating the scores based on the Q/As in the excel

In [28]:
# Excel file path
file_path = "Arabic_QA_Benchmark_qwen.xlsx"

# Extract questions and answers from the Excel file
questions, gold_answers = extract_questions_and_answers(file_path, question_col=3, answer_col=5, start_row=3)
questions, llm_answers = extract_questions_and_answers(file_path, question_col=3, answer_col=6, start_row=3)

In [29]:
# Load a multilingual SBERT model that supports Arabic
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# Calculate STS scores
sts_scores = get_STS_score(llm_answers, gold_answers, model)

Candidate: <think>
ال пользователя question هو عن أكبر قارة في العالم بمساحة. أولاً، يجب أن أتذكر المعلومات الأساسية عن القارات. أعرف أن آسيا هي أكبر قارة لكنني أريد التأكد من الرقم الدقيق. آسيا تغطي حوالي 44 مليون كيلومترا مربّعا، وهي أكبر من الأفريقيّة التي تأتي في المرتبة الثانية. يجب أن أتأكد أيضاً من أنني لا أخلط بين المساحة الإجمالية والمساحة المأهولة. لا، السؤال عن المساحة الكلية. ربما أذكر też fact أن القارة القطبية الجنوبيّة تأتي بعد ذلك، لكن عندما نتحدث عن القارات السبع، آسيا明明 هي الأولى. هل هناك أي تغييرات حديثة في حدود القارات؟ لا أعتقد ذلك. الإجابة الصحيحة هي آسيا. يجب الإجابة مباشرة كما يطلب السؤال، دون معلومات إضافية.
</think>

آسيا هي أكبر قارة في العالم من حيث المساحة، حيث تغطي مساحة تประมาณ 44 مليون كيلومتر مربع، وهو ما يعادل ما يقرب من 30% من مساحة اليابسة في العالم و8.7% من مساحة كوكب الأرض الكلية.
Reference: قارة آسيا
STS score: 0.6625

Candidate: <think>
Okay, the user is asking "ما هو أطول نهر في العالم؟" which means "What is the longest river in the world?" in A

In [25]:
# get BERTScore
P, R, F1 = get_bert_score(llm_answers, gold_answers, lang="ar", model_type="xlm-roberta-large")

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 100.52 seconds, 1.19 sentences/sec
Candidate: 亚细亚 هي أكبر قارة في العالم من حيث المساحة.
Reference: قارة آسيا
BERTScore F1: 0.8721553087234497
———
Candidate: نيل!
Reference: نهر النيل
BERTScore F1: 0.8693298101425171
———
Candidate: تونس، الجزائر، المغرب، مصر، ليبيا، السودان، إيطاليا، اليونان، تركيا، إسرائيل، قبرص، سوريا، لبنان.
Reference: في أوروبا:
إسبانيا
فرنسا
موناكو
إيطاليا
سلوفينيا
كرواتيا
البوسنة والهرسك
الجبل الأسود
ألبانيا
اليونان
قبرص (جغرافياً في آسيا لكنها غالباً تُدرج أوروبياً)

 في آسيا:
تركيا
سوريا
لبنان
فلسطين (قطاع غزة يطل على البحر)

 في أفريقيا:
مصر
ليبيا
تونس
الجزائر
المغرب
BERTScore F1: 0.8545746207237244
———
Candidate: دول شمال أفريقيا هي المغرب والجزائر وتونس وليبيا ومصر.
Reference: مصر
ليبيا
تونس
الجزائر
المغرب
موريتانيا (غالبًا تُدرج ضمن شمال غرب أفريقيا)
BERTScore F1: 0.8710601329803467
———
Candidate: جغرافيا بتهتم بدراسة الأرض والظواهر الطبيعيةوالانسانية الموجودة عليها، زي الاودية والجبال والبحار والمحيطات والشعوب واللغات والثقافات والمدن والقرى والطرق

In [26]:
print("BERTScore F1: ", F1)
len(F1)

BERTScore F1:  tensor([0.8722, 0.8693, 0.8546, 0.8711, 0.8733, 0.8679, 0.8570, 0.8959, 0.8691,
        0.8568, 0.8922, 0.8822, 0.9014, 0.8535, 0.8517, 0.9182, 0.8679, 0.8616,
        0.8483, 0.8359, 0.8349, 0.7831, 0.8792, 0.8661, 0.8646, 0.9454, 0.8297,
        0.8461, 0.8589, 0.8711, 0.8618, 0.9612, 0.8612, 0.8745, 0.8752, 0.8125,
        0.8560, 0.8970, 0.8691, 0.8776, 0.8860, 0.8679, 0.8821, 0.8595, 0.8374,
        0.8492, 0.9676, 0.8512, 0.8761, 0.8524, 0.8519, 0.8681, 0.8621, 0.8612,
        0.8414, 0.8947, 0.9528, 0.8755, 0.8658, 0.8524, 0.8659, 0.8712, 0.8643,
        0.9191, 0.8701, 0.8928, 0.8690, 0.8638, 0.8627, 0.8193, 0.8281, 0.8993,
        0.8786, 0.8822, 0.8593, 0.8557, 0.8444, 0.8782, 0.8712, 0.8616, 0.7423,
        0.8855, 0.8470, 0.8663, 0.8641, 0.8544, 0.8162, 0.8719, 0.8759, 0.8563,
        0.8571, 0.8121, 0.7675, 0.8256, 0.8597, 0.9127, 0.8828, 0.8755, 0.8753,
        0.8600, 0.8682, 0.8903, 0.8588, 0.8783, 0.8676, 0.8648, 0.9130, 0.8621,
        0.8689, 0.9011, 0

120

In [27]:
# Get average BERTScore F1
avg_bert_score = sum(F1) / len(F1)
print(f"Average BERTScore F1: {avg_bert_score:.4f}")

Average BERTScore F1: 0.8662
